## Calculate and save global integrals
This notebook calculates the global integral of biogeochemical quantities and saves them to netcdf files here: `/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals/`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import esm2m.config_perturbations as cp
import os
from dask.diagnostics import ProgressBar

In [ ]:
# dmget files
cp.dmget_exps(ppname='bling_atm_flux',wait=False,verbose=True)

In [34]:
grid = cp.load_grid()

### Global integrals (x,y,z)

In [14]:
variables = cp.disp_variables()
experiments = ['','_gat','_zero','_double']
override=False
# Directory to save file in
outroot = '/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals/'
if override:
    outroot = outroot+'override-po4/'
else:
    outroot = outroot+'no-override/'
    
for variable in variables:
    # Ignore variables that have already been integrated
    if 'integral' in variable:
        continue
    for e in experiments:
        outfile = 'esm2m_'+variable+e+'.nc'
        outpath = outroot+outfile
        if os.path.isfile(outpath):
            print(outfile+' exists.')
            continue
        print(outpath)
        # Load dataset
        ds = cp.load_exps(variable=variable,override=override,experiments=e)
        # Isolate variable
        da = ds[variable]
        dadims = list(da.dims)
        daattrs = da.attrs.copy()
        if 'st_ocean' in dadims:
            factor = 1035.0 * grid['volume_t']
            daattrs['units'] = daattrs['units']+' * kg'
        else:
            factor = grid['area_t']
            daattrs['units'] = daattrs['units']+' * m2'
        # Remove time from list of dimensions before summing
        dadims.remove('time')
        dasum = ((da*factor).sum(dadims)
                 .assign_attrs(daattrs))
        dasum.name = variable
        
        with ProgressBar():
            dasum.to_dataset().to_netcdf(outpath)

esm2m_alk.nc exists.
esm2m_alk_gat.nc exists.
esm2m_alk_zero.nc exists.
esm2m_alk_double.nc exists.
esm2m_alpha.nc exists.
esm2m_alpha_gat.nc exists.
esm2m_alpha_zero.nc exists.
esm2m_alpha_double.nc exists.
esm2m_biomass_p.nc exists.
esm2m_biomass_p_gat.nc exists.
esm2m_biomass_p_zero.nc exists.
esm2m_biomass_p_double.nc exists.
esm2m_chl.nc exists.
esm2m_chl_gat.nc exists.
esm2m_chl_zero.nc exists.
esm2m_chl_double.nc exists.
esm2m_co2_alpha.nc exists.
esm2m_co2_alpha_gat.nc exists.
esm2m_co2_alpha_zero.nc exists.
esm2m_co2_alpha_double.nc exists.
esm2m_co3_ion.nc exists.
esm2m_co3_ion_gat.nc exists.
esm2m_co3_ion_zero.nc exists.
esm2m_co3_ion_double.nc exists.
esm2m_delta_csurf.nc exists.
esm2m_delta_csurf_gat.nc exists.
esm2m_delta_csurf_zero.nc exists.
esm2m_delta_csurf_double.nc exists.
esm2m_delta_pco2.nc exists.
esm2m_delta_pco2_gat.nc exists.
esm2m_delta_pco2_zero.nc exists.
esm2m_delta_pco2_double.nc exists.
esm2m_dic.nc exists.
esm2m_dic_gat.nc exists.
esm2m_dic_zero.nc exis

### Global integrals, depth-resolved (x,y)

In [ ]:
variables = cp.disp_variables()
experiments = ['','_gat','_zero','_double']
override=False
# Directory to save file in
outroot = '/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/'
outroot = cp.add_override_suffix(outroot,override)
    
for variable in variables:
    # Ignore variables that have already been integrated
    if ('integral' in variable) or ('flux' in variable) or ('stf' in variable):
        continue
    for e in experiments:
        outfile = 'esm2m_'+variable+e+'.nc'
        outpath = outroot+outfile
        if os.path.isfile(outpath):
            print(outfile+' exists.')
            continue
        print(outpath)
        
        # Load dataset
        ds = cp.load_exps(variable=variable,override=override,experiments=e)
        # Isolate variable
        da = ds[variable]
        dadims = list(da.dims)
        daattrs = da.attrs.copy()
        if ('st_ocean' not in dadims):
            print(variable+' is 2D.')
            continue
        else:
            factor = 1035.0 * grid['volume_t']
            daattrs['units'] = daattrs['units']+' * kg'
            
        # Remove time from list of dimensions before summing
        dadims.remove('time')
        dadims.remove('st_ocean')
        dasum = ((da*factor).sum(dadims)
                 .assign_attrs(daattrs))
        dasum.name = variable
        
        with ProgressBar():
            dasum.to_dataset().to_netcdf(outpath)

/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk.nc
[########################################] | 100% Completed | 22.3s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_gat.nc
[########################################] | 100% Completed | 23.0s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_zero.nc
[########################################] | 100% Completed | 25.9s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_double.nc
[########################################] | 100% Completed | 22.9s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alpha.nc
[########################################] | 100% Completed | 22.1s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alpha_g

In [ ]:
variable = 'co2_flux'
if 'integral' in variable or 'flux' in variable:
    print('True')